Instalación

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils
!pip install openai
!pip install flash-attn

In [ ]:
import torch
if torch.cuda.is_available():
    print("GPU está disponible:", torch.cuda.get_device_name(0))
else:
    print("GPU NO está disponible.")

Descargar una imagen

In [ ]:
!pip install wget
import wget
from PIL import Image

url = "https://cdn.resfu.com/media/img_news/zinedine-zidane--en-un-partido-de-champions-league-con-el-real-madrid.jpg?size=1000x&lossy=1"
img = wget.download(url)
imagen = Image.open(img)


# Show the image
from IPython.display import Image
Image(filename=img)



In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
checkpoint = "Qwen/Qwen2.5-VL-7B-Instruct"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")#, attn_implementation="flash_attention_2") #attn_implementation="flash_attention_2"
processor = AutoProcessor.from_pretrained(checkpoint)


In [ ]:
import base64



def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

prompt = 'Describe the given image', # Ajustar el prompt a las necesidades


In [ ]:
messages=[
          {
          "role": "system",
          "content": [{"type":"text","text": "You are a helpful assistant."}]
          },

          {
          "role": "user",
          "content": [
              {
                  "type": "image_url",
                  "image_url": {"url": f"data:image/jpeg;base64,{encode_image(img)}"},
              },
              {"type": "text", "text": prompt},
          ],
          }
  ]

text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)



inputs = processor(text=[text], images = [imagen],  padding=True, return_tensors="pt")
inputs = inputs.to('cuda')

output_ids = model.generate(**inputs, max_new_tokens=4096)
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

print(output_text[0])
